<a href="https://colab.research.google.com/github/LC1332/Haruhi-2-Dev/blob/main/notebook/%E9%A2%9D%E5%A4%96%E4%B8%AD%E6%96%87%E6%95%B0%E6%8D%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] RoleLLM的数据
- [ ] 其他中文的hf的一些数据
- [ ] ChatHaruhi54K本身的数据

这里我们假设输入数据已经组织成了hf格式

可以有效载入到chatbot中

In [1]:
!pip -q install openai tiktoken langchain chromadb datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

训练的参数设定

In [33]:
K_SEARCH = 5
MAX_LEN_STORY = 1000 #这个是按照token算的
MAX_LEN_HISTORY = 1200 # count with token

In [34]:
import os
import openai
key = "key is not necessary here"
key_bytes = key.encode()
os.environ["OPENAI_API_KEY"] = key_bytes.decode('utf-8')

In [35]:
agent_datas = []

role_names = ['令狐冲', '岳不群', '鸠摩智', '萧峰', '丁春秋', '虚竹', '谢逊', '张无忌', '赵敏', '周芷若', '黄药师', '欧阳锋', '黄蓉', '郭靖', '孙悟空']

hf_roles = [
    "chengli-thu/linghuchong",
    "chengli-thu/yuebuqun",
    "hhhwmws/jiumozhi",
    "hhhwmws/xiaofeng",
    "hhhwmws/dingchunqiu",
    "hhhwmws/xuzhu",
    "hhhwmws/xiexun",
    "hhhwmws/zhangwuji",
    "hhhwmws/zhaomin",
    "hhhwmws/zhouzhiruo",
    "hhhwmws/huangyaoshi",
    "hhhwmws/ouyangfeng",
    "hhhwmws/huangrong",
    "hhhwmws/guojing",
    "sibozhu/wukong"
]

for role, hf_add in zip(role_names, hf_roles):
    agent_data = {
        "hf_str":hf_add,
        "role":role,
        "embedding":"luotuo_openai"
    }
    agent_datas.append(agent_data)


In [20]:
role_from_roleLLM = ['Caesar', 'Sonny', 'Angel', 'Jigsaw', 'John Doe', 'Freddy Krueger', 'Colonel Hans Landa', 'Gregory House', 'Gaston', 'HAL 9000', 'Mark Renton', 'Coriolanus', 'Oliver Queen', 'Bruno Antony', 'D_Artagnan', 'Dr. Frank-N-Furter', 'Tugg Speedman', 'Stifler', 'Jeff Spicoli', 'Rorschach', 'Paul Vitti', 'Logan', 'Judge Dredd', 'Karl Childers', 'Rachel Lang', 'Queen Elizabeth I', 'Tyrion Lannister', 'John Keating', 'Wade Wilson', 'Lyn Cassady', 'Dr. Hannibal Lecter', 'Violet Weston', 'Po', 'Malcolm X', 'Willie Soke', 'Jack Torrance', 'Alvy Singer', 'Colonel Nathan R. Jessep', 'Andrew Detmer', 'Fred Flintstone', 'Frank T.J. Mackey', 'Stephen Hawking', 'Lestat de Lioncourt', 'Jack Sparrow', 'John Coffey', 'Murphy MacManus', 'John Dillinger', 'Jackie Moon', 'Peter Parker', 'Abraham Lincoln', 'James Carter', 'Tyler Hawkins', 'Stanley Ipkiss', 'Mater', 'Professor G.H. Dorr', 'Juno MacGuff', 'Seth', 'Sherlock Holmes', 'Truman Capote', 'Shrek', 'Travis Bickle', 'Jack', 'Tom Ripley', 'The Dude', 'David Aames', 'Twilight Sparkle', 'Antonio Salieri', 'Judy Hoops', 'Randle McMurphy', 'Thor', 'Walt Kowalski', 'Fletcher Reede', 'Theodore Twombly', 'James Brown', 'Paul Conroy', 'James Bond', 'Queen Catherine', 'Harvey Milk', 'Caden Cotard', 'Leonard Shelby', 'Jim Morrison', 'Pat Solitano', 'Benjamin Button', 'Robert Angier', 'Lucifer Morningstar', 'Jordan Belfort', 'Coach Eric Taylor', 'Mary Sibley', 'Klaus Mikaelson', 'Raylan Givens', 'Sheldon Cooper', 'Michael Scott', 'Leroy Jethro Gibbs', 'Doctor Who', 'Blair Waldorf']
print(len(role_from_roleLLM))

# # for debug
# role_from_roleLLM = role_from_roleLLM[:5]
# print(len(role_from_roleLLM))

95


In [21]:

for role in role_from_roleLLM:
    agent_data = {
        "hf_str":'silk-road/ChatHaruhi-from-RoleLLM/' + role,
        "role":role,
        "embedding":"bge_en"
    }
    agent_datas.append(agent_data)

In [22]:
print(len(agent_datas))

110


In [7]:

%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev

/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 898, done.
remote: Counting objects: 100% (205/205), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 898 (delta 138), reused 152 (delta 105), pack-reused 693
Receiving objects: 100% (898/898), 105.85 MiB | 16.36 MiB/s, done.
Resolving deltas: 100% (468/468), done.
/content/Haruhi-2-Dev


In [8]:
!ls

ChatHaruhi  data  LICENSE  notebook  Readme.md


In [23]:
from ChatHaruhi import ChatHaruhi

from ChatHaruhi.ChromaDB import ChromaDB

class ChatHaruhiTrain(ChatHaruhi):

    def build_story_db_from_vec( self, texts, vecs ):
        self.db = ChromaDB()
        self.stories = texts
        self.db.init_from_docs( vecs, texts)

    def add_story_with_expire(self, query, expire_story):
        if self.db is None:
            print("No vec DB！")
            return

        query_vec = self.embedding(query)
        stories = self.db.search(query_vec, self.k_search)

        story_string = self.story_prefix_prompt + self.dialogue_divide_token

        sum_story_token = self.tokenizer(story_string)

        for story in stories:
            if expire_story.strip() == story.strip():
                continue

            story_token = self.tokenizer(story.strip()) + self.tokenizer(self.dialogue_divide_token)
            if sum_story_token + story_token > self.max_len_story:
                break
            else:
                sum_story_token += story_token
                story_string += story.strip() + self.dialogue_divide_token

        self.llm.user_message(story_string)

    def generate_prompt(self, query, history_str, expire_story ):
        # 这里修改下其他超参，不规范删了
        # self.k_search = 5
        # self.max_len_story = 1500
        # self.max_len_history = 1200
        self.story_prefix_prompt = "\nClassic scenes for the role are as follows:"

        self.llm.initialize_message()

        self.llm.system_message(self.system_prompt)

        self.add_story_with_expire(query, expire_story)

        # self.add_history(history)
        history_message = self.dialogue_divide_token + history_str
        self.llm.user_message(history)

        self.llm.user_message(query)

        # self.llm.user_message(target)

        return self.llm.messages

    def add_history(self, history_list):

        if len(history_list) == 0:
            return

        sum_history_token = 0
        flag = 0
        for history in history_list:
            current_count = 0
            if history is not None:
                current_count += self.tokenizer(history)

            sum_history_token += current_count
            if sum_history_token > self.max_len_history:
                break
            else:
                flag += 1

        if flag == 0:
            print('warning! no history added. the last dialogue is too long.')

        # 是否添加历史前缀，
        history_message = ""
        for history in history_list[-flag:]:
            history_message += history
        self.llm.user_message(history_message)

In [28]:
def extract_tuples_from_story(role_name, story):
    story_tuples = []

    lines = story.split('\n')

    n = len(lines)

    datas = []

    for i in range(1, n):
        line = lines[i]
        if line.startswith(role_name):
            query = lines[i-1]
            target = line
            history_str = ""
            for j in range(0, i-1):
                history_str += lines[j] + "\n"

            history_str = history_str.strip()
            data = (query, target, history_str, story)
            datas.append(data)

    return datas

对RoleLLM的数据进行组织

In [25]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [26]:
print(agent_datas[0])

{'hf_str': 'chengli-thu/linghuchong', 'role': '令狐冲', 'embedding': 'luotuo_openai'}


In [ ]:
from datasets import load_dataset
from tqdm import tqdm

save_datas = []

# for role_name in tqdm(role_from_roleLLM):

for agent_data in agent_datas:
    role_name = agent_data['role']
    hf_address = agent_data['hf_str']
    embedding = agent_data['embedding']

    chatbot = ChatHaruhiTrain( role_from_hf = hf_address, \
                      llm = 'openai',
                      embedding = embedding)
    stories = chatbot.stories

    chatbot.k_search = K_SEARCH
    chatbot.max_len_story = MAX_LEN_STORY
    chatbot.max_len_history = MAX_LEN_HISTORY

    all_tuples = []

    for story in stories:
        # print(story)
        tuples = extract_tuples_from_story(role_name, story )

        all_tuples += tuples

    for query, target, history, story in tqdm(all_tuples):
        messages = chatbot.generate_prompt(query, history, story)
        # print(prompt)

        prompt = ""

        for msg in messages:
            if isinstance(msg, HumanMessage):
                prompt += msg.content + "\n"
            elif isinstance(msg, AIMessage):
                prompt += msg.content + "\n"
            elif isinstance(msg, SystemMessage):
                prompt += msg.content + "\n"

        save_data = {
            'context': prompt,
            'target': target
        }

        save_datas.append(save_data)

        # break

    # break

 18%|█▊        | 490/2747 [02:43<12:11,  3.09it/s]

In [ ]:
print(save_data['context'])

print()

print(save_data['target'])

save_datas是一个list of dict
save_name = "/content/ChatHaruhi_rolellm_fusion.jsonl"

帮我用utf-8编码，jsonl格式，保存save_datas中的数据
ensure_ascii = False

In [ ]:
print(len(save_datas))

In [ ]:
import json

save_name = "/content/ChatHaruhi_Chinese_New_Data.jsonl"

with open(save_name, 'w', encoding='utf8') as f:
    for data in save_datas:
        json.dump(data, f, ensure_ascii=False)
        f.write('\n')